Based on:
- https://www.kaggle.com/ashishpatel26/preprocessing-lightgbm-xgboost
- https://github.com/Shitao-zz/Kaggle-House-Prices-Advanced-Regression-Techniques
- https://www.kaggle.com/vinicius150987/regression-boston-sklearn-dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import swifter

from category_encoders import HelmertEncoder, SumEncoder, OneHotEncoder
#from fancyimpute import KNN, SoftImpute, BiScaler, IterativeSVD, MatrixFactorization, SimpleFill

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.model_selection import KFold
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
from sklearn.datasets import load_boston, make_classification, make_regression
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.linear_model import BayesianRidge, SGDRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, median_absolute_error

pd.options.mode.chained_assignment = None 
pd.options.display.max_columns = None

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
cat_cols = ['Neutered/Spayed', 'Gender_f', 'MIXED_BREED_FLAG', 'BREED', 'neuter_dt_bool']
num_cols = ['ALB', 'ALKP', 'ALT', 'AMYL', 'BUN', 'CA', 'CHOL', 'CREA', 'EOSINOPHIL', 'GLOB', 'GLU',
       'HCT', 'HGB', 'LYMPHOCYTE', 'MCH', 'MCHC', 'MCV', 'MONOCYTE', 'MPV',
       'PHOS', 'PLT', 'RBC', 'RDW', 'TBIL', 'TP', 'WBC']
target = ['VST_AGE_YEARS']
len(cat_cols) + len(num_cols)

31

In [6]:
encode_cat = False
scaling = True

save_data = False
#drop_nan_rows = False
#drop_error_data = False

random_state = 42

#imputer_num = IterativeImputer(estimator=SGDRegressor(loss='huber', penalty='l1'), verbose=2, max_iter=1)
#imputer_num = IterativeImputer(estimator=BayesianRidge(), verbose=2, max_iter=30)
#imputer_num = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, criterion="mae", n_jobs=-1, random_state=random_state), verbose=2, max_iter=5, n_nearest_features=3)
#imputer_num = IterativeImputer(estimator=KNeighborsRegressor(n_jobs=-1), n_nearest_features=5, verbose=2, max_iter=5)
imputer_num = SimpleImputer()
#imputer_num = KNNImputer(n_neighbors=5) # also slow https://github.com/scikit-learn/scikit-learn/issues/18186, 33min for 200k rows TODO: normalize before! but gives error :(

imputer_cat = SimpleImputer(strategy='most_frequent')

scaler = StandardScaler()
encoder = OneHotEncoder(cat_cols)

In [7]:
def save_memory(df, compress_float=True):
    features = df.columns
    for i in range( df.shape[1] ):
        if df.dtypes[i] == 'uint8':
            df[features[i]] = df[features[i]].astype( np.int8 )
        elif df.dtypes[i] == 'bool':
            df[features[i]] = df[features[i]].astype( np.int8 )
        elif df.dtypes[i] == 'uint32':
            df[features[i]] = df[features[i]].astype( np.int32 )
        elif df.dtypes[i] == 'int64':
            df[features[i]] = df[features[i]].astype( np.int32 )
        elif df.dtypes[i] == 'float64' and compress_float:
            df[features[i]] = df[features[i]].astype( np.float32 )

In [3]:
df = pd.read_csv('data/data_original.csv')
df.head(5)

,DW_PET_VST_ID,VST_AGE_YEARS,GENDER,MIXED_BREED_FLAG,BREED,neuter_dt,VST_DT,DW_PET_ID,correct_age_flag,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
0,121909452,1.150,Spayed Female,2.0,DLH,2014-01-25T09:18:25Z,2014-07-25,33891682,0.0,3.6,75.0,79.0,839.0,32.0,9.6,92.0,2.1,NaN,4.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,NaN,NaN,0.4,7.6,NaN
1,125735678,8.562,Neutered Male,2.0,DSH,NaN,2014-11-11,12440683,0.0,2.6,31.0,61.0,992.0,20.0,9.5,162.0,1.5,NaN,4.8,142.0,45.139999,133.0,NaN,14.11,29.370001,48.000000,NaN,10.27,3.0,351.0,9.40,14.180000,0.8,7.4,4.81
2,126172184,6.800,Neutered Male,2.0,DSH,NaN,2014-11-24,16531176,0.0,3.5,35.0,85.0,698.0,21.0,10.3,105.0,2.1,NaN,5.2,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,NaN,NaN,0.1,8.7,NaN
3,85691964,1.380,Spayed Female,2.0,DLH,NaN,2011-08-18,24553376,0.0,3.5,59.0,70.0,668.0,23.0,10.2,203.0,1.6,9.3,3.6,114.0,42.000000,146.0,33.599998,17.10,34.700001,49.299999,4.5,10.50,5.0,332.0,8.53,18.299999,0.1,7.1,6.97
4,152742000,0.490,Spayed Female,0.0,Calico,2016-11-28T07:17:52Z,2016-11-28,38286369,1.0,3.3,97.0,39.0,533.0,23.0,11.0,133.0,1.1,3.2,3.1,92.0,NaN,134.0,38.299999,14.40,NaN,NaN,1.5,NaN,8.2,554.0,9.30,19.900000,0.1,6.4,10.28


In [8]:
# remove dog data 
df = df[~df['BREED'].isin(['Akita', 'Ardennes Cattle Dog', 'Australian Cattle Dog', 'Australian Kelpie', 'Shih Tzu', 'Terrier, Yorkshire'])]

# remove possible bad data
# may want to keep flag == 1 after closer inspection
df = df[(df.correct_age_flag != 1) & (df.correct_age_flag != 2)]

# remove inf values
df = df.replace([np.inf, -np.inf], np.nan)

# neuter dt to flag
df['neuter_dt_bool'] = df['neuter_dt'].notnull().astype(bool)

# change data types
df['MIXED_BREED_FLAG'] = df['MIXED_BREED_FLAG'].astype('Int64')

# split up gender
df[['Neutered/Spayed', 'Gender']] = df['GENDER'].str.split(' ', 1, expand=True)
df.loc[df['Neutered/Spayed'] == 'Female', 'Gender'] = 'Female'
df.loc[df['Neutered/Spayed'] == 'Male', 'Gender'] = 'Male'
df.loc[df['Neutered/Spayed'] == 'Female', 'Neutered/Spayed'] = 'None'
df.loc[df['Neutered/Spayed'] == 'Male', 'Neutered/Spayed'] = 'None'

df['Gender'] = df['Gender'].replace({
    "Female": 1, 
    "Male": 0
    })

df['Gender_f'] = df['Gender'].astype(bool)
df = df.drop('Gender', axis=1)

for cat in cat_cols:
    df[cat] = df[cat].astype("category")

# replace string nan 
df = df.replace('nan', np.nan)

df = df[target + cat_cols + num_cols]

print(df.shape)
print(df.dtypes)
df.head()

(801681, 32)
VST_AGE_YEARS        float64
Neutered/Spayed     category
Gender_f            category
MIXED_BREED_FLAG    category
BREED               category
neuter_dt_bool      category
ALB                  float64
ALKP                 float64
ALT                  float64
AMYL                 float64
BUN                  float64
CA                   float64
CHOL                 float64
CREA                 float64
EOSINOPHIL           float64
GLOB                 float64
GLU                  float64
HCT                  float64
HGB                  float64
LYMPHOCYTE           float64
MCH                  float64
MCHC                 float64
MCV                  float64
MONOCYTE             float64
MPV                  float64
PHOS                 float64
PLT                  float64
RBC                  float64
RDW                  float64
TBIL                 float64
TP                   float64
WBC                  float64
dtype: object


,VST_AGE_YEARS,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
0,1.150,Spayed,True,2,DLH,True,3.6,75.0,79.0,839.0,32.0,9.6,92.0,2.1,NaN,4.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,NaN,NaN,0.4,7.6,NaN
1,8.562,Neutered,False,2,DSH,False,2.6,31.0,61.0,992.0,20.0,9.5,162.0,1.5,NaN,4.8,142.0,45.139999,133.0,NaN,14.11,29.370001,48.000000,NaN,10.27,3.0,351.0,9.40,14.180000,0.8,7.4,4.81
2,6.800,Neutered,False,2,DSH,False,3.5,35.0,85.0,698.0,21.0,10.3,105.0,2.1,NaN,5.2,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,NaN,NaN,0.1,8.7,NaN
3,1.380,Spayed,True,2,DLH,False,3.5,59.0,70.0,668.0,23.0,10.2,203.0,1.6,9.3,3.6,114.0,42.000000,146.0,33.599998,17.10,34.700001,49.299999,4.5,10.50,5.0,332.0,8.53,18.299999,0.1,7.1,6.97
5,0.370,Spayed,True,2,DSH,True,3.3,176.0,37.0,676.0,22.0,11.0,112.0,1.3,NaN,3.3,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,0.1,6.6,NaN


In [9]:
df_test = pd.read_csv('data/data_test.csv')

# remove dog data 
df_test = df_test[~df_test['BREED'].isin(['Akita', 'Ardennes Cattle Dog', 'Australian Cattle Dog', 'Australian Kelpie', 'Shih Tzu', 'Terrier, Yorkshire'])]

# remove inf values
df_test = df_test.replace([np.inf, -np.inf], np.nan)

# neuter dt to flag
df_test['neuter_dt_bool'] = df_test['neuter_dt'].notnull().astype(bool)

# change data types
df_test['MIXED_BREED_FLAG'] = df_test['MIXED_BREED_FLAG'].astype('Int64')

# split up gender
df_test[['Neutered/Spayed', 'Gender']] = df_test['GENDER'].str.split(' ', 1, expand=True)
df_test.loc[df_test['Neutered/Spayed'] == 'Female', 'Gender'] = 'Female'
df_test.loc[df_test['Neutered/Spayed'] == 'Male', 'Gender'] = 'Male'
df_test.loc[df_test['Neutered/Spayed'] == 'Female', 'Neutered/Spayed'] = 'None'
df_test.loc[df_test['Neutered/Spayed'] == 'Male', 'Neutered/Spayed'] = 'None'

df_test['Gender'] = df_test['Gender'].replace({
    "Female": 1, 
    "Male": 0
    })

df_test['Gender_f'] = df_test['Gender'].astype(bool)
df_test = df_test.drop('Gender', axis=1)

for cat in cat_cols:
    df_test[cat] = df_test[cat].astype("category")

# replace string nan 
df_test = df_test.replace('nan', np.nan)

X_test = df_test[['DW_PET_VST_ID'] + cat_cols + num_cols]

print(X_test.shape)
print(X_test.dtypes)
X_test.head()

(140844, 32)
DW_PET_VST_ID          int64
Neutered/Spayed     category
Gender_f            category
MIXED_BREED_FLAG    category
BREED               category
neuter_dt_bool      category
ALB                  float64
ALKP                 float64
ALT                  float64
AMYL                 float64
BUN                  float64
CA                   float64
CHOL                 float64
CREA                 float64
EOSINOPHIL           float64
GLOB                 float64
GLU                  float64
HCT                  float64
HGB                  float64
LYMPHOCYTE           float64
MCH                  float64
MCHC                 float64
MCV                  float64
MONOCYTE             float64
MPV                  float64
PHOS                 float64
PLT                  float64
RBC                  float64
RDW                  float64
TBIL                 float64
TP                   float64
WBC                  float64
dtype: object


,DW_PET_VST_ID,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
0,76818738,Neutered,False,2,DLH,True,2.5,36.0,23.0,718.0,23.0,9.9,158.0,1.1,NaN,4.4,103.0,37.560001,120.599998,NaN,15.57,32.110001,48.0,6.5,9.99,5.7,75.0,7.75,15.880000,0.1,7.0,9.17
1,115189677,Spayed,True,2,DMH,True,3.6,51.0,60.0,863.0,24.0,9.7,111.0,1.4,6.0,3.9,102.0,41.310001,137.199997,28.0,15.70,33.220001,47.0,3.0,9.30,3.8,74.0,8.74,16.290001,0.1,7.5,7.64
2,140822898,Spayed,True,0,Bombay,False,3.3,18.0,73.0,981.0,27.0,10.0,143.0,1.8,0.0,4.7,120.0,43.680000,142.399994,50.0,16.42,32.599998,50.0,10.0,8.30,4.9,NaN,8.67,15.240000,1.5,8.0,9.11
3,137576910,Neutered,False,2,DSH,False,3.5,37.0,59.0,1169.0,26.0,10.4,220.0,1.9,NaN,3.7,95.0,33.240002,93.800003,NaN,13.27,28.209999,47.0,NaN,10.12,5.8,738.0,7.06,14.660000,0.3,7.2,7.04
4,68211647,Spayed,True,2,DSH,False,2.8,141.0,25.0,554.0,22.0,10.0,308.0,1.9,NaN,5.1,126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.5,7.8,NaN


In [10]:
X = df.drop(target, axis=1)
y = df[target]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=random_state)
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
X_train.head()

(721512, 31) (721512, 1) (80169, 31) (80169, 1)


,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
304181,Spayed,True,2,DSH,False,3.4,53.0,164.0,818.0,21.0,10.9,169.0,1.5,NaN,4.9,124.0,49.630001,157.399994,NaN,16.469999,31.719999,52.0,0.0,9.52,4.4,73.0,9.56,14.42,0.8,8.3,10.15
841211,Spayed,True,1,Siamese,True,2.8,172.0,80.0,621.0,22.0,10.5,105.0,0.8,NaN,3.5,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3,NaN,NaN,NaN,0.1,6.3,NaN
1038584,Spayed,True,2,DSH,True,4.0,37.0,51.0,622.0,19.0,10.3,189.0,1.5,0.0,3.7,119.0,39.830002,125.000000,53.900002,16.230000,31.389999,52.0,2.5,9.38,4.3,136.0,7.70,14.20,0.4,7.7,4.63
624702,Neutered,False,2,DSH,False,3.4,32.0,30.0,1545.0,23.0,9.9,77.0,1.6,0.0,4.3,110.0,32.919998,109.699997,39.000000,15.160000,33.330002,45.0,8.0,10.00,4.0,NaN,7.24,17.42,0.5,7.6,6.25
127303,Spayed,True,0,Siamese,False,2.8,35.0,99.0,670.0,22.0,9.8,167.0,1.5,3.5,4.0,148.0,NaN,131.000000,19.700001,14.100000,NaN,NaN,1.3,NaN,2.2,164.0,9.31,18.90,0.1,6.7,8.80


In [11]:
# Imputation
X_train[num_cols] = imputer_num.fit_transform(X_train[num_cols])
X_valid[num_cols] = imputer_num.transform(X_valid[num_cols])
X_test[num_cols] = imputer_num.transform(X_test[num_cols])

X_train[cat_cols] = imputer_cat.fit_transform(X_train[cat_cols])
X_valid[cat_cols] = imputer_cat.transform(X_valid[cat_cols])
X_test[cat_cols] = imputer_cat.transform(X_test[cat_cols])

X_train.head()

,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
304181,Spayed,True,2,DSH,False,3.4,53.0,164.0,818.0,21.0,10.9,169.0,1.5,3.283438,4.9,124.0,49.630001,157.399994,31.060026,16.469999,31.719999,52.000000,0.000000,9.520000,4.4,73.000000,9.560000,14.420000,0.8,8.3,10.150000
841211,Spayed,True,1,Siamese,True,2.8,172.0,80.0,621.0,22.0,10.5,105.0,0.8,3.283438,3.5,92.0,41.103224,133.092489,31.060026,15.504562,32.029558,48.121409,4.046114,10.241562,7.3,296.239684,8.697005,15.763807,0.1,6.3,7.904972
1038584,Spayed,True,2,DSH,True,4.0,37.0,51.0,622.0,19.0,10.3,189.0,1.5,0.000000,3.7,119.0,39.830002,125.000000,53.900002,16.230000,31.389999,52.000000,2.500000,9.380000,4.3,136.000000,7.700000,14.200000,0.4,7.7,4.630000
624702,Neutered,False,2,DSH,False,3.4,32.0,30.0,1545.0,23.0,9.9,77.0,1.6,0.000000,4.3,110.0,32.919998,109.699997,39.000000,15.160000,33.330002,45.000000,8.000000,10.000000,4.0,296.239684,7.240000,17.420000,0.5,7.6,6.250000
127303,Spayed,True,0,Siamese,False,2.8,35.0,99.0,670.0,22.0,9.8,167.0,1.5,3.500000,4.0,148.0,41.103224,131.000000,19.700001,14.100000,32.029558,48.121409,1.300000,10.241562,2.2,164.000000,9.310000,18.900000,0.1,6.7,8.800000


In [12]:
# NORMALIZE/STANDARDIZE
if scaling:
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])

X_train.head()

,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
304181,Spayed,True,2,DSH,False,0.565117,-0.083936,3.298568,0.055539,-0.923631,1.045984,0.639321,-0.119074,3.928145e-16,1.405175,0.291423,0.981102,7.352667e-01,-1.054413e-15,6.288612e-02,-4.646169e-02,4.715268e-01,-1.400820,-4.903107e-01,-0.593610,-1.944974e+00,0.079278,-2.674955e-01,0.801551,1.394026,1.218854e-01
841211,Spayed,True,1,Siamese,True,-1.014025,3.139680,0.555261,-0.959741,-0.690719,0.441009,-1.186900,-1.000346,3.928145e-16,-1.126764,-0.937075,0.000000,8.597155e-16,-1.054413e-15,4.628292e-16,1.066452e-15,1.727638e-15,0.000000,1.207058e-15,1.676055,1.485744e-15,0.000000,2.121587e-15,-0.530343,-1.410314,4.822035e-17
1038584,Spayed,True,2,DSH,True,2.144259,-0.517364,-0.391833,-0.954587,-1.389455,0.138522,1.210015,-0.119074,-9.681104e-01,-0.765058,0.099470,-0.146499,-2.447860e-01,2.259566e+00,4.725315e-02,-9.599132e-02,4.715268e-01,-0.535286,-5.854428e-01,-0.671874,-1.396087e+00,-0.091589,-3.112883e-01,0.040468,0.552723,-1.778024e-01
624702,Neutered,False,2,DSH,False,0.565117,-0.652810,-1.077660,3.802281,-0.457807,-0.466454,-1.985872,0.006822,-9.681104e-01,0.320059,-0.246045,-0.941572,-7.075889e-01,7.855043e-01,-2.244386e-02,1.951833e-01,-3.794750e-01,1.368889,-1.641447e-01,-0.906667,1.485744e-15,-0.133847,3.296782e-01,0.230739,0.412507,-8.985056e-02
127303,Spayed,True,0,Siamese,False,-1.014025,-0.571542,1.175771,-0.707210,-0.690719,-0.617697,0.582251,-0.119074,6.385263e-02,-0.222500,1.212797,0.000000,-6.329475e-02,-1.123851e+00,-9.148949e-02,1.066452e-15,1.727638e-15,-0.950742,1.207058e-15,-2.315424,-1.152137e+00,0.056312,6.242838e-01,-0.530343,-0.849447,4.859223e-02


In [13]:
# One Hot Encoding
if encode_cat:
    X_train = encoder.fit_transform(X_train, y_train)
    X_valid = encoder.transform(X_valid)
    X_test = encoder.transform(X_test)
else:
    for cat in cat_cols:
        X_train[cat] = X_train[cat].astype("category")
        X_valid[cat] = X_valid[cat].astype("category")
        X_test[cat] = X_test[cat].astype("category")
        print(f"{cat} {X_test[cat].dtype}")
y_train = y_train.astype("float32")
y_valid = y_valid.astype("float32")
X_train.head()

Neutered/Spayed category
Gender_f category
MIXED_BREED_FLAG category
BREED category
neuter_dt_bool category


,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
304181,Spayed,True,2,DSH,False,0.565117,-0.083936,3.298568,0.055539,-0.923631,1.045984,0.639321,-0.119074,3.928145e-16,1.405175,0.291423,0.981102,7.352667e-01,-1.054413e-15,6.288612e-02,-4.646169e-02,4.715268e-01,-1.400820,-4.903107e-01,-0.593610,-1.944974e+00,0.079278,-2.674955e-01,0.801551,1.394026,1.218854e-01
841211,Spayed,True,1,Siamese,True,-1.014025,3.139680,0.555261,-0.959741,-0.690719,0.441009,-1.186900,-1.000346,3.928145e-16,-1.126764,-0.937075,0.000000,8.597155e-16,-1.054413e-15,4.628292e-16,1.066452e-15,1.727638e-15,0.000000,1.207058e-15,1.676055,1.485744e-15,0.000000,2.121587e-15,-0.530343,-1.410314,4.822035e-17
1038584,Spayed,True,2,DSH,True,2.144259,-0.517364,-0.391833,-0.954587,-1.389455,0.138522,1.210015,-0.119074,-9.681104e-01,-0.765058,0.099470,-0.146499,-2.447860e-01,2.259566e+00,4.725315e-02,-9.599132e-02,4.715268e-01,-0.535286,-5.854428e-01,-0.671874,-1.396087e+00,-0.091589,-3.112883e-01,0.040468,0.552723,-1.778024e-01
624702,Neutered,False,2,DSH,False,0.565117,-0.652810,-1.077660,3.802281,-0.457807,-0.466454,-1.985872,0.006822,-9.681104e-01,0.320059,-0.246045,-0.941572,-7.075889e-01,7.855043e-01,-2.244386e-02,1.951833e-01,-3.794750e-01,1.368889,-1.641447e-01,-0.906667,1.485744e-15,-0.133847,3.296782e-01,0.230739,0.412507,-8.985056e-02
127303,Spayed,True,0,Siamese,False,-1.014025,-0.571542,1.175771,-0.707210,-0.690719,-0.617697,0.582251,-0.119074,6.385263e-02,-0.222500,1.212797,0.000000,-6.329475e-02,-1.123851e+00,-9.148949e-02,1.066452e-15,1.727638e-15,-0.950742,1.207058e-15,-2.315424,-1.152137e+00,0.056312,6.242838e-01,-0.530343,-0.849447,4.859223e-02


In [14]:
X_test.head(10)

,DW_PET_VST_ID,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC
0,76818738,Neutered,False,2,DLH,True,-1.803596,-0.544453,-1.306269,-0.459832,-0.457807,-0.466454,0.325439,-0.622658,3.928145e-16,0.500911,-0.514779,-0.407688,-3.778797e-01,-1.054413e-15,4.262467e-03,1.207354e-02,-1.475993e-02,0.849569,-1.709400e-01,0.423826,-1.927549e+00,-0.086996,2.312906e-02,-0.530343,-0.428795,6.868000e-02
1,115189677,Spayed,True,2,DMH,True,1.091497,-0.138115,-0.097908,0.287455,-0.224895,-0.768941,-1.015692,-0.244970,8.009691e-01,-0.403353,-0.553170,0.023792,1.242462e-01,-3.027293e-01,1.273034e-02,1.786734e-01,-1.363316e-01,-0.362179,-6.398039e-01,-1.063196,-1.936262e+00,0.003950,1.047430e-01,-0.530343,0.272290,-1.438568e-02
2,140822898,Spayed,True,0,Bombay,False,0.301926,-1.032059,0.326652,0.895592,0.473842,-0.315209,-0.102582,0.258614,-9.681104e-01,1.043469,0.137861,0.296487,2.815385e-01,1.873738e+00,5.962930e-02,8.561724e-02,2.283835e-01,2.061316,-1.319317e+00,-0.202288,1.485744e-15,-0.002481,-1.042681e-01,2.133445,0.973375,6.542250e-02
3,137576910,Neutered,False,2,DSH,False,0.828307,-0.517364,-0.130566,1.864488,0.240930,0.289765,2.094590,0.384510,3.928145e-16,-0.765058,-0.821904,-0.904752,-1.188541e+00,-1.054413e-15,-1.455535e-01,-5.732770e-01,-1.363316e-01,0.000000,-8.260316e-02,0.502090,3.848834e+00,-0.150382,-2.197216e-01,-0.149802,-0.148362,-4.696045e-02
4,68211647,Spayed,True,2,DSH,False,-1.014025,2.299914,-1.240952,-1.305039,-0.690719,-0.315209,4.605644,0.384510,3.928145e-16,1.766880,0.368204,0.000000,8.597155e-16,-1.054413e-15,4.628292e-16,1.066452e-15,1.727638e-15,0.000000,1.207058e-15,-0.906667,1.485744e-15,0.000000,2.121587e-15,0.230739,0.692941,4.822035e-17
5,136237223,Neutered,False,2,DLH,True,-1.014025,-0.273561,-0.457150,0.308070,0.473842,-1.071428,1.267084,0.510406,-9.681104e-01,-0.403353,-0.361217,0.210191,-1.056425e-01,1.234945e+00,-7.325103e-02,-2.760989e-01,-1.475993e-02,1.230403,-1.366884e+00,-1.454517,-5.335503e-01,0.029672,-1.281550e-01,-0.149802,-0.849447,-1.355761e-03
6,130928278,Spayed,True,2,DSH,False,-1.540406,-0.354828,0.359310,-1.052508,0.939666,-0.920184,-1.158366,-0.496762,3.928145e-16,1.586028,0.483376,0.782046,6.021733e-01,3.699969e-01,1.625464e-01,-4.436428e-03,2.173531e+00,-0.639150,3.794661e-01,-1.611046,8.517361e-01,-0.018098,7.204442e-03,0.040468,0.412507,-1.794312e-01
7,93115144,Spayed,True,2,DSH,True,0.565117,-0.815345,0.130701,0.220457,-0.457807,0.592254,1.095876,-0.119074,-9.681104e-01,-0.403353,-1.090638,0.177974,-7.841902e-02,-8.508254e-02,2.640920e-02,-2.130612e-01,4.715268e-01,-0.743014,-1.672664e+00,-0.358817,-2.093087e+00,-0.045657,-4.884497e-01,0.611280,-0.008144,-2.320937e-01
8,96301433,Spayed,True,2,DSH,False,1.617878,0.593294,0.653236,-1.408113,-0.457807,-0.163965,0.011557,0.132718,3.928145e-16,-0.945911,-0.860294,0.214794,1.968430e-01,-1.054413e-15,-1.409939e-01,6.910784e-02,-8.657618e-01,0.000000,1.228858e+00,0.423826,2.088910e+00,0.162875,5.213818e-03,0.230739,0.132073,3.037613e-01
9,139205328,Neutered,False,2,DSH,True,-0.487644,0.782919,-0.293858,-0.717517,0.473842,0.592254,-0.188186,-1.000346,3.928145e-16,-0.765058,-1.013856,0.000000,8.597155e-16,-1.054413e-15,4.628292e-16,1.066452e-15,1.727638e-15,0.000000,1.207058e-15,2.771754,1.485744e-15,0.000000,2.121587e-15,-0.149802,-0.849447,4.822035e-17


In [19]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, objective="regression_l1",
            boosting_type='dart',
            learning_rate=0.13110563910426518,
            max_depth=50,
            n_estimators=1000,
            num_leaves=200,
            reg_alpha=1.5921278701590583e-05,
            reg_lambda=1.0,
            subsample_for_bin=200000
)
model.fit(X_train, y_train)

print("Train MAE: ", mean_absolute_error(y_train, np.abs(model.predict(X_train))))
print("MeanAE: ", mean_absolute_error(y_valid, np.abs(model.predict(X_valid))))
print("MedianAE: ", median_absolute_error(y_valid, np.abs(model.predict(X_valid))))

Train MAE:  1.8552827107874696
MeanAE:  2.0121393167334927
MedianAE:  1.355405031455346


In [20]:
#raise SystemExit("Stop right there!")
np.abs(model.predict(X_test.drop('DW_PET_VST_ID', axis=1)))

array([1.88516536, 2.85980285, 3.6265916 , ..., 5.89383178, 6.61827982,
       1.82287848])

In [15]:
save_memory(X_train, compress_float=False)
save_memory(X_test, compress_float=False)
print(X_train.dtypes)

if encode_cat:
    file_name = "_pre_simple.csv"
else:
    file_name = "_pre_simple_no_enc.csv"

if save_data:
    X_train.to_csv('data/X_train' + file_name, index=False)
    X_valid.to_csv('data/X_valid' + file_name, index=False)
    X_test.to_csv('data/X_test' + file_name, index=False)
    y_train.to_csv('data/y_train' + file_name, index=False)
    y_valid.to_csv('data/y_valid' + file_name, index=False)

Neutered/Spayed     category
Gender_f            category
MIXED_BREED_FLAG    category
BREED               category
neuter_dt_bool      category
ALB                  float64
ALKP                 float64
ALT                  float64
AMYL                 float64
BUN                  float64
CA                   float64
CHOL                 float64
CREA                 float64
EOSINOPHIL           float64
GLOB                 float64
GLU                  float64
HCT                  float64
HGB                  float64
LYMPHOCYTE           float64
MCH                  float64
MCHC                 float64
MCV                  float64
MONOCYTE             float64
MPV                  float64
PHOS                 float64
PLT                  float64
RBC                  float64
RDW                  float64
TBIL                 float64
TP                   float64
WBC                  float64
dtype: object


In [16]:
flist = num_cols
X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

# perform a polynomial features transform of the dataset
trans = PolynomialFeatures(degree=2) # exponential! only 2 or 3
trans_train = trans.fit_transform(X_train[flist])
trans_valid = trans.transform(X_valid[flist])
trans_test = trans.transform(X_test[flist])

X_train = pd.concat([X_train, pd.DataFrame(trans_train, columns=trans.get_feature_names(X_train[flist].columns))], axis=1)
X_valid = pd.concat([X_valid, pd.DataFrame(trans_valid, columns=trans.get_feature_names(X_valid[flist].columns))], axis=1)
X_test = pd.concat([X_test, pd.DataFrame(trans_test, columns=trans.get_feature_names(X_test[flist].columns))], axis=1)

print(X_train.shape, y_train.shape)
X_train.head()

(721512, 409) (721512, 1)


,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,1,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,ALB^2,ALB ALKP,ALB ALT,ALB AMYL,ALB BUN,ALB CA,ALB CHOL,ALB CREA,ALB EOSINOPHIL,ALB GLOB,ALB GLU,ALB HCT,ALB HGB,ALB LYMPHOCYTE,ALB MCH,ALB MCHC,ALB MCV,ALB MONOCYTE,ALB MPV,ALB PHOS,ALB PLT,ALB RBC,ALB RDW,ALB TBIL,ALB TP,ALB WBC,ALKP^2,ALKP ALT,ALKP AMYL,ALKP BUN,ALKP CA,ALKP CHOL,ALKP CREA,ALKP EOSINOPHIL,ALKP GLOB,ALKP GLU,ALKP HCT,ALKP HGB,ALKP LYMPHOCYTE,ALKP MCH,ALKP MCHC,ALKP MCV,ALKP MONOCYTE,ALKP MPV,ALKP PHOS,ALKP PLT,ALKP RBC,ALKP RDW,ALKP TBIL,ALKP TP,ALKP WBC,ALT^2,ALT AMYL,ALT BUN,ALT CA,ALT CHOL,ALT CREA,ALT EOSINOPHIL,ALT GLOB,ALT GLU,ALT HCT,ALT HGB,ALT LYMPHOCYTE,ALT MCH,ALT MCHC,ALT MCV,ALT MONOCYTE,ALT MPV,ALT PHOS,ALT PLT,ALT RBC,ALT RDW,ALT TBIL,ALT TP,ALT WBC,AMYL^2,AMYL BUN,AMYL CA,AMYL CHOL,AMYL CREA,AMYL EOSINOPHIL,AMYL GLOB,AMYL GLU,AMYL HCT,AMYL HGB,AMYL LYMPHOCYTE,AMYL MCH,AMYL MCHC,AMYL MCV,AMYL MONOCYTE,AMYL MPV,AMYL PHOS,AMYL PLT,AMYL RBC,AMYL RDW,AMYL TBIL,AMYL TP,AMYL WBC,BUN^2,BUN CA,BUN CHOL,BUN CREA,BUN EOSINOPHIL,BUN GLOB,BUN GLU,BUN HCT,BUN HGB,BUN LYMPHOCYTE,BUN MCH,BUN MCHC,BUN MCV,BUN MONOCYTE,BUN MPV,BUN PHOS,BUN PLT,BUN RBC,BUN RDW,BUN TBIL,BUN TP,BUN WBC,CA^2,CA CHOL,CA CREA,CA EOSINOPHIL,CA GLOB,CA GLU,CA HCT,CA HGB,CA LYMPHOCYTE,CA MCH,CA MCHC,CA MCV,CA MONOCYTE,CA MPV,CA PHOS,CA PLT,CA RBC,CA RDW,CA TBIL,CA TP,CA WBC,CHOL^2,CHOL CREA,CHOL EOSINOPHIL,CHOL GLOB,CHOL GLU,CHOL HCT,CHOL HGB,CHOL LYMPHOCYTE,CHOL MCH,CHOL MCHC,CHOL MCV,CHOL MONOCYTE,CHOL MPV,CHOL PHOS,CHOL PLT,CHOL RBC,CHOL RDW,CHOL TBIL,CHOL TP,CHOL WBC,CREA^2,CREA EOSINOPHIL,CREA GLOB,CREA GLU,CREA HCT,CREA HGB,CREA LYMPHOCYTE,CREA MCH,CREA MCHC,CREA MCV,CREA MONOCYTE,CREA MPV,CREA PHOS,CREA PLT,CREA RBC,CREA RDW,CREA TBIL,CREA TP,CREA WBC,EOSINOPHIL^2,EOSINOPHIL GLOB,EOSINOPHIL GLU,EOSINOPHIL HCT,EOSINOPHIL HGB,EOSINOPHIL LYMPHOCYTE,EOSINOPHIL MCH,EOSINOPHIL MCHC,EOSINOPHIL MCV,EOSINOPHIL MONOCYTE,EOSINOPHIL MPV,EOSINOPHIL PHOS,EOSINOPHIL PLT,EOSINOPHIL RBC,EOSINOPHIL RDW,EOSINOPHIL TBIL,EOSINOPHIL TP,EOSINOPHIL WBC,GLOB^2,GLOB GLU,GLOB HCT,GLOB HGB,GLOB LYMPHOCYTE,GLOB MCH,GLOB MCHC,GLOB MCV,GLOB MONOCYTE,GLOB MPV,GLOB PHOS,GLOB PLT,GLOB RBC,GLOB RDW,GLOB TBIL,GLOB TP,GLOB WBC,GLU^2,GLU HCT,GLU HGB,GLU LYMPHOCYTE,GLU MCH,GLU MCHC,GLU MCV,GLU MONOCYTE,GLU MPV,GLU PHOS,GLU PLT,GLU RBC,GLU RDW,GLU TBIL,GLU TP,GLU WBC,HCT^2,HCT HGB,HCT LYMPHOCYTE,HCT MCH,HCT MCHC,HCT MCV,HCT MONOCYTE,HCT MPV,HCT PHOS,HCT PLT,HCT RBC,HCT RDW,HCT TBIL,HCT TP,HCT WBC,HGB^2,HGB LYMPHOCYTE,HGB MCH,HGB MCHC,HGB MCV,HGB MONOCYTE,HGB MPV,HGB PHOS,HGB PLT,HGB RBC,HGB RDW,HGB TBIL,HGB TP,HGB WBC,LYMPHOCYTE^2,LYMPHOCYTE MCH,LYMPHOCYTE MCHC,LYMPHOCYTE MCV,LYMPHOCYTE MONOCYTE,LYMPHOCYTE MPV,LYMPHOCYTE PHOS,LYMPHOCYTE PLT,LYMPHOCYTE RBC,LYMPHOCYTE RDW,LYMPHOCYTE TBIL,LYMPHOCYTE TP,LYMPHOCYTE WBC,MCH^2,MCH MCHC,MCH MCV,MCH MONOCYTE,MCH MPV,MCH PHOS,MCH PLT,MCH RBC,MCH RDW,MCH TBIL,MCH TP,MCH WBC,MCHC^2,MCHC MCV,MCHC MONOCYTE,MCHC MPV,MCHC PHOS,MCHC PLT,MCHC RBC,MCHC RDW,MCHC TBIL,MCHC TP,MCHC WBC,MCV^2,MCV MONOCYTE,MCV MPV,MCV PHOS,MCV PLT,MCV RBC,MCV RDW,MCV TBIL,MCV TP,MCV WBC,MONOCYTE^2,MONOCYTE MPV,MONOCYTE PHOS,MONOCYTE PLT,MONOCYTE RBC,MONOCYTE RDW,MONOCYTE TBIL,MONOCYTE TP,MONOCYTE WBC,MPV^2,MPV PHOS,MPV PLT,MPV RBC,MPV RDW,MPV TBIL,MPV TP,MPV WBC,PHOS^2,PHOS PLT,PHOS RBC,PHOS RDW,PHOS TBIL,PHOS TP,PHOS WBC,PLT^2,PLT RBC,PLT RDW,PLT TBIL,PLT TP,PLT WBC,RBC^2,RBC RDW,RBC TBIL,RBC TP,RBC WBC,RDW^2,RDW TBIL,RDW TP,RDW WBC,TBIL^2,TBIL TP,TBIL WBC,TP^2,TP WBC,WBC^2
0,Spayed,True,2,DSH,False,0.565117,-0.083936,3.298568,0.055539,-0.923631,1.045984,0.639321,-0.119074,3.928145e-16,1.405175,0.291423,0.981102,7.352667e-01,-1.054413e-15,6.288612e-02,-4.646169e-02,4.715268e-01,-1.400820,-4.903107e-01,-0.593610,-1.944974e+00,0.079278,-2.67

In [17]:
def kmeans(X_tr, X_vl, X_te, flist):
    flist_kmeans = []
    for ncl in range(2,11):
        cls = KMeans(n_clusters=ncl)
        cls.fit_predict(X_train[flist].values)
        X_tr['kmeans_cluster_'+str(ncl)] = cls.predict(X_tr[flist].values)
        X_vl['kmeans_cluster_'+str(ncl)] = cls.predict(X_vl[flist].values)
        X_te['kmeans_cluster_'+str(ncl)] = cls.predict(X_te[flist].values)
        flist_kmeans.append('kmeans_cluster_'+str(ncl))
    print(flist_kmeans)
    
    return X_tr, X_vl, X_te

In [18]:
flist = num_cols
X_train, X_valid, X_test = kmeans(X_train, X_valid, X_test, flist)
X_train.head()

['kmeans_cluster_2', 'kmeans_cluster_3', 'kmeans_cluster_4', 'kmeans_cluster_5', 'kmeans_cluster_6', 'kmeans_cluster_7', 'kmeans_cluster_8', 'kmeans_cluster_9', 'kmeans_cluster_10']


,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,1,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,ALB^2,ALB ALKP,ALB ALT,ALB AMYL,ALB BUN,ALB CA,ALB CHOL,ALB CREA,ALB EOSINOPHIL,ALB GLOB,ALB GLU,ALB HCT,ALB HGB,ALB LYMPHOCYTE,ALB MCH,ALB MCHC,ALB MCV,ALB MONOCYTE,ALB MPV,ALB PHOS,ALB PLT,ALB RBC,ALB RDW,ALB TBIL,ALB TP,ALB WBC,ALKP^2,ALKP ALT,ALKP AMYL,ALKP BUN,ALKP CA,ALKP CHOL,ALKP CREA,ALKP EOSINOPHIL,ALKP GLOB,ALKP GLU,ALKP HCT,ALKP HGB,ALKP LYMPHOCYTE,ALKP MCH,ALKP MCHC,ALKP MCV,ALKP MONOCYTE,ALKP MPV,ALKP PHOS,ALKP PLT,ALKP RBC,ALKP RDW,ALKP TBIL,ALKP TP,ALKP WBC,ALT^2,ALT AMYL,ALT BUN,ALT CA,ALT CHOL,ALT CREA,ALT EOSINOPHIL,ALT GLOB,ALT GLU,ALT HCT,ALT HGB,ALT LYMPHOCYTE,ALT MCH,ALT MCHC,ALT MCV,ALT MONOCYTE,ALT MPV,ALT PHOS,ALT PLT,ALT RBC,ALT RDW,ALT TBIL,ALT TP,ALT WBC,AMYL^2,AMYL BUN,AMYL CA,AMYL CHOL,AMYL CREA,AMYL EOSINOPHIL,AMYL GLOB,AMYL GLU,AMYL HCT,AMYL HGB,AMYL LYMPHOCYTE,AMYL MCH,AMYL MCHC,AMYL MCV,AMYL MONOCYTE,AMYL MPV,AMYL PHOS,AMYL PLT,AMYL RBC,AMYL RDW,AMYL TBIL,AMYL TP,AMYL WBC,BUN^2,BUN CA,BUN CHOL,BUN CREA,BUN EOSINOPHIL,BUN GLOB,BUN GLU,BUN HCT,BUN HGB,BUN LYMPHOCYTE,BUN MCH,BUN MCHC,BUN MCV,BUN MONOCYTE,BUN MPV,BUN PHOS,BUN PLT,BUN RBC,BUN RDW,BUN TBIL,BUN TP,BUN WBC,CA^2,CA CHOL,CA CREA,CA EOSINOPHIL,CA GLOB,CA GLU,CA HCT,CA HGB,CA LYMPHOCYTE,CA MCH,CA MCHC,CA MCV,CA MONOCYTE,CA MPV,CA PHOS,CA PLT,CA RBC,CA RDW,CA TBIL,CA TP,CA WBC,CHOL^2,CHOL CREA,CHOL EOSINOPHIL,CHOL GLOB,CHOL GLU,CHOL HCT,CHOL HGB,CHOL LYMPHOCYTE,CHOL MCH,CHOL MCHC,CHOL MCV,CHOL MONOCYTE,CHOL MPV,CHOL PHOS,CHOL PLT,CHOL RBC,CHOL RDW,CHOL TBIL,CHOL TP,CHOL WBC,CREA^2,CREA EOSINOPHIL,CREA GLOB,CREA GLU,CREA HCT,CREA HGB,CREA LYMPHOCYTE,CREA MCH,CREA MCHC,CREA MCV,CREA MONOCYTE,CREA MPV,CREA PHOS,CREA PLT,CREA RBC,CREA RDW,CREA TBIL,CREA TP,CREA WBC,EOSINOPHIL^2,EOSINOPHIL GLOB,EOSINOPHIL GLU,EOSINOPHIL HCT,EOSINOPHIL HGB,EOSINOPHIL LYMPHOCYTE,EOSINOPHIL MCH,EOSINOPHIL MCHC,EOSINOPHIL MCV,EOSINOPHIL MONOCYTE,EOSINOPHIL MPV,EOSINOPHIL PHOS,EOSINOPHIL PLT,EOSINOPHIL RBC,EOSINOPHIL RDW,EOSINOPHIL TBIL,EOSINOPHIL TP,EOSINOPHIL WBC,GLOB^2,GLOB GLU,GLOB HCT,GLOB HGB,GLOB LYMPHOCYTE,GLOB MCH,GLOB MCHC,GLOB MCV,GLOB MONOCYTE,GLOB MPV,GLOB PHOS,GLOB PLT,GLOB RBC,GLOB RDW,GLOB TBIL,GLOB TP,GLOB WBC,GLU^2,GLU HCT,GLU HGB,GLU LYMPHOCYTE,GLU MCH,GLU MCHC,GLU MCV,GLU MONOCYTE,GLU MPV,GLU PHOS,GLU PLT,GLU RBC,GLU RDW,GLU TBIL,GLU TP,GLU WBC,HCT^2,HCT HGB,HCT LYMPHOCYTE,HCT MCH,HCT MCHC,HCT MCV,HCT MONOCYTE,HCT MPV,HCT PHOS,HCT PLT,HCT RBC,HCT RDW,HCT TBIL,HCT TP,HCT WBC,HGB^2,HGB LYMPHOCYTE,HGB MCH,HGB MCHC,HGB MCV,HGB MONOCYTE,HGB MPV,HGB PHOS,HGB PLT,HGB RBC,HGB RDW,HGB TBIL,HGB TP,HGB WBC,LYMPHOCYTE^2,LYMPHOCYTE MCH,LYMPHOCYTE MCHC,LYMPHOCYTE MCV,LYMPHOCYTE MONOCYTE,LYMPHOCYTE MPV,LYMPHOCYTE PHOS,LYMPHOCYTE PLT,LYMPHOCYTE RBC,LYMPHOCYTE RDW,LYMPHOCYTE TBIL,LYMPHOCYTE TP,LYMPHOCYTE WBC,MCH^2,MCH MCHC,MCH MCV,MCH MONOCYTE,MCH MPV,MCH PHOS,MCH PLT,MCH RBC,MCH RDW,MCH TBIL,MCH TP,MCH WBC,MCHC^2,MCHC MCV,MCHC MONOCYTE,MCHC MPV,MCHC PHOS,MCHC PLT,MCHC RBC,MCHC RDW,MCHC TBIL,MCHC TP,MCHC WBC,MCV^2,MCV MONOCYTE,MCV MPV,MCV PHOS,MCV PLT,MCV RBC,MCV RDW,MCV TBIL,MCV TP,MCV WBC,MONOCYTE^2,MONOCYTE MPV,MONOCYTE PHOS,MONOCYTE PLT,MONOCYTE RBC,MONOCYTE RDW,MONOCYTE TBIL,MONOCYTE TP,MONOCYTE WBC,MPV^2,MPV PHOS,MPV PLT,MPV RBC,MPV RDW,MPV TBIL,MPV TP,MPV WBC,PHOS^2,PHOS PLT,PHOS RBC,PHOS RDW,PHOS TBIL,PHOS TP,PHOS WBC,PLT^2,PLT RBC,PLT RDW,PLT TBIL,PLT TP,PLT WBC,RBC^2,RBC RDW,RBC TBIL,RBC TP,RBC WBC,RDW^2,RDW TBIL,RDW TP,RDW WBC,TBIL^2,TBIL TP,TBIL WBC,TP^2,TP WBC,WBC^2,kmeans_cluster_2,kmeans_cluster_3,kmeans_cluster_4,kmeans_cluster_5,kmeans_cluster_6,kmeans_cluster_7,kmeans_cluster_8,kmeans_cluster_9,kmeans_cluster_10
0,Spayed,True,2,DSH,False,0.565117,-0.083936,3.298568,0.055539,-0.923631,1.045984,0.639321,-0.119074,3.928145e-16,1.

In [19]:
N_COMP = 20           ### Number of decomposition components ###
flist = num_cols

print("\nStart decomposition process...")
print("PCA")
pca = PCA(n_components=N_COMP, random_state=random_state)
pca_results_train = pca.fit_transform(X_train[flist].values)
pca_results_valid = pca.transform(X_valid[flist].values)
pca_results_test = pca.transform(X_test[flist].values)
print(pca.explained_variance_ratio_)

print("tSVD")
tsvd = TruncatedSVD(n_components=N_COMP, random_state=random_state)
tsvd_results_train = tsvd.fit_transform(X_train[flist].values)
tsvd_results_valid = tsvd.transform(X_valid[flist].values)
tsvd_results_test = tsvd.transform(X_test[flist].values)

print("ICA")
ica = FastICA(n_components=N_COMP, random_state=random_state)
ica_results_train = ica.fit_transform(X_train[flist].values)
ica_results_valid = ica.transform(X_valid[flist].values)
ica_results_test = ica.transform(X_test[flist].values)

print("GRP")
grp = GaussianRandomProjection(n_components=N_COMP, eps=0.1, random_state=random_state)
grp_results_train = grp.fit_transform(X_train[flist].values)
grp_results_valid = grp.transform(X_valid[flist].values)
grp_results_test = grp.transform(X_test[flist].values)

print("SRP")
srp = SparseRandomProjection(n_components=N_COMP, dense_output=True, random_state=random_state)
srp_results_train = srp.fit_transform(X_train[flist].values)
srp_results_valid = srp.transform(X_valid[flist].values)
srp_results_test = srp.transform(X_test[flist].values)

print("Append decomposition components to datasets...")
for i in range(1, N_COMP + 1):
    X_train['pca_' + str(i)] = pca_results_train[:, i - 1]
    X_valid['pca_' + str(i)] = pca_results_valid[:, i - 1]
    X_test['pca_' + str(i)] = pca_results_test[:, i - 1]
    
    X_train['ica_' + str(i)] = ica_results_train[:, i - 1]
    X_valid['ica_' + str(i)] = ica_results_valid[:, i - 1]
    X_test['ica_' + str(i)] = ica_results_test[:, i - 1]

    X_train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    X_valid['tsvd_' + str(i)] = tsvd_results_valid[:, i - 1]
    X_test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    X_train['grp_' + str(i)] = grp_results_train[:, i - 1]
    X_valid['grp_' + str(i)] = grp_results_valid[:, i - 1]
    X_test['grp_' + str(i)] = grp_results_test[:, i - 1]

    X_train['srp_' + str(i)] = srp_results_train[:, i - 1]
    X_valid['srp_' + str(i)] = srp_results_valid[:, i - 1]
    X_test['srp_' + str(i)] = srp_results_test[:, i - 1]
    
print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))
X_train.head()


Start decomposition process...
PCA
[0.11400715 0.09262482 0.07387389 0.05219391 0.04395089 0.04321289
 0.04154814 0.04021176 0.03969144 0.03895503 0.03815295 0.03767823
 0.03713119 0.03577296 0.03496261 0.03408403 0.03344271 0.0306731
 0.02932436 0.0276819 ]
tSVD
ICA
GRP
SRP
Append decomposition components to datasets...

Train shape: (721512, 518)
Test shape: (140844, 519)


,Neutered/Spayed,Gender_f,MIXED_BREED_FLAG,BREED,neuter_dt_bool,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,1,ALB,ALKP,ALT,AMYL,BUN,CA,CHOL,CREA,EOSINOPHIL,GLOB,GLU,HCT,HGB,LYMPHOCYTE,MCH,MCHC,MCV,MONOCYTE,MPV,PHOS,PLT,RBC,RDW,TBIL,TP,WBC,ALB^2,ALB ALKP,ALB ALT,ALB AMYL,ALB BUN,ALB CA,ALB CHOL,ALB CREA,ALB EOSINOPHIL,ALB GLOB,ALB GLU,ALB HCT,ALB HGB,ALB LYMPHOCYTE,ALB MCH,ALB MCHC,ALB MCV,ALB MONOCYTE,ALB MPV,ALB PHOS,ALB PLT,ALB RBC,ALB RDW,ALB TBIL,ALB TP,ALB WBC,ALKP^2,ALKP ALT,ALKP AMYL,ALKP BUN,ALKP CA,ALKP CHOL,ALKP CREA,ALKP EOSINOPHIL,ALKP GLOB,ALKP GLU,ALKP HCT,ALKP HGB,ALKP LYMPHOCYTE,ALKP MCH,ALKP MCHC,ALKP MCV,ALKP MONOCYTE,ALKP MPV,ALKP PHOS,ALKP PLT,ALKP RBC,ALKP RDW,ALKP TBIL,ALKP TP,ALKP WBC,ALT^2,ALT AMYL,ALT BUN,ALT CA,ALT CHOL,ALT CREA,ALT EOSINOPHIL,ALT GLOB,ALT GLU,ALT HCT,ALT HGB,ALT LYMPHOCYTE,ALT MCH,ALT MCHC,ALT MCV,ALT MONOCYTE,ALT MPV,ALT PHOS,ALT PLT,ALT RBC,ALT RDW,ALT TBIL,ALT TP,ALT WBC,AMYL^2,AMYL BUN,AMYL CA,AMYL CHOL,AMYL CREA,AMYL EOSINOPHIL,AMYL GLOB,AMYL GLU,AMYL HCT,AMYL HGB,AMYL LYMPHOCYTE,AMYL MCH,AMYL MCHC,AMYL MCV,AMYL MONOCYTE,AMYL MPV,AMYL PHOS,AMYL PLT,AMYL RBC,AMYL RDW,AMYL TBIL,AMYL TP,AMYL WBC,BUN^2,BUN CA,BUN CHOL,BUN CREA,BUN EOSINOPHIL,BUN GLOB,BUN GLU,BUN HCT,BUN HGB,BUN LYMPHOCYTE,BUN MCH,BUN MCHC,BUN MCV,BUN MONOCYTE,BUN MPV,BUN PHOS,BUN PLT,BUN RBC,BUN RDW,BUN TBIL,BUN TP,BUN WBC,CA^2,CA CHOL,CA CREA,CA EOSINOPHIL,CA GLOB,CA GLU,CA HCT,CA HGB,CA LYMPHOCYTE,CA MCH,CA MCHC,CA MCV,CA MONOCYTE,CA MPV,CA PHOS,CA PLT,CA RBC,CA RDW,CA TBIL,CA TP,CA WBC,CHOL^2,CHOL CREA,CHOL EOSINOPHIL,CHOL GLOB,CHOL GLU,CHOL HCT,CHOL HGB,CHOL LYMPHOCYTE,CHOL MCH,CHOL MCHC,CHOL MCV,CHOL MONOCYTE,CHOL MPV,CHOL PHOS,CHOL PLT,CHOL RBC,CHOL RDW,CHOL TBIL,CHOL TP,CHOL WBC,CREA^2,CREA EOSINOPHIL,CREA GLOB,CREA GLU,CREA HCT,CREA HGB,CREA LYMPHOCYTE,CREA MCH,CREA MCHC,CREA MCV,CREA MONOCYTE,CREA MPV,CREA PHOS,CREA PLT,CREA RBC,CREA RDW,CREA TBIL,CREA TP,CREA WBC,EOSINOPHIL^2,EOSINOPHIL GLOB,EOSINOPHIL GLU,EOSINOPHIL HCT,EOSINOPHIL HGB,EOSINOPHIL LYMPHOCYTE,EOSINOPHIL MCH,EOSINOPHIL MCHC,EOSINOPHIL MCV,EOSINOPHIL MONOCYTE,EOSINOPHIL MPV,EOSINOPHIL PHOS,EOSINOPHIL PLT,EOSINOPHIL RBC,EOSINOPHIL RDW,EOSINOPHIL TBIL,EOSINOPHIL TP,EOSINOPHIL WBC,GLOB^2,GLOB GLU,GLOB HCT,GLOB HGB,GLOB LYMPHOCYTE,GLOB MCH,GLOB MCHC,GLOB MCV,GLOB MONOCYTE,GLOB MPV,GLOB PHOS,GLOB PLT,GLOB RBC,GLOB RDW,GLOB TBIL,GLOB TP,GLOB WBC,GLU^2,GLU HCT,GLU HGB,GLU LYMPHOCYTE,GLU MCH,GLU MCHC,GLU MCV,GLU MONOCYTE,GLU MPV,GLU PHOS,GLU PLT,GLU RBC,GLU RDW,GLU TBIL,GLU TP,GLU WBC,HCT^2,HCT HGB,HCT LYMPHOCYTE,HCT MCH,HCT MCHC,HCT MCV,HCT MONOCYTE,HCT MPV,HCT PHOS,HCT PLT,HCT RBC,HCT RDW,HCT TBIL,HCT TP,HCT WBC,HGB^2,HGB LYMPHOCYTE,HGB MCH,HGB MCHC,HGB MCV,HGB MONOCYTE,HGB MPV,HGB PHOS,HGB PLT,HGB RBC,HGB RDW,HGB TBIL,HGB TP,HGB WBC,LYMPHOCYTE^2,LYMPHOCYTE MCH,LYMPHOCYTE MCHC,LYMPHOCYTE MCV,LYMPHOCYTE MONOCYTE,LYMPHOCYTE MPV,LYMPHOCYTE PHOS,LYMPHOCYTE PLT,LYMPHOCYTE RBC,LYMPHOCYTE RDW,LYMPHOCYTE TBIL,LYMPHOCYTE TP,LYMPHOCYTE WBC,MCH^2,MCH MCHC,MCH MCV,MCH MONOCYTE,MCH MPV,MCH PHOS,MCH PLT,MCH RBC,MCH RDW,MCH TBIL,MCH TP,MCH WBC,MCHC^2,MCHC MCV,MCHC MONOCYTE,MCHC MPV,MCHC PHOS,MCHC PLT,MCHC RBC,MCHC RDW,MCHC TBIL,MCHC TP,MCHC WBC,MCV^2,MCV MONOCYTE,MCV MPV,MCV PHOS,MCV PLT,MCV RBC,MCV RDW,MCV TBIL,MCV TP,MCV WBC,MONOCYTE^2,MONOCYTE MPV,MONOCYTE PHOS,MONOCYTE PLT,MONOCYTE RBC,MONOCYTE RDW,MONOCYTE TBIL,MONOCYTE TP,MONOCYTE WBC,MPV^2,MPV PHOS,MPV PLT,MPV RBC,MPV RDW,MPV TBIL,MPV TP,MPV WBC,PHOS^2,PHOS PLT,PHOS RBC,PHOS RDW,PHOS TBIL,PHOS TP,PHOS WBC,PLT^2,PLT RBC,PLT RDW,PLT TBIL,PLT TP,PLT WBC,RBC^2,RBC RDW,RBC TBIL,RBC TP,RBC WBC,RDW^2,RDW TBIL,RDW TP,RDW WBC,TBIL^2,TBIL TP,TBIL WBC,TP^2,TP WBC,WBC^2,kmeans_cluster_2,kmeans_cluster_3,kmeans_cluster_4,kmeans_cluster_5,kmeans_cluster_6,kmeans_cluster_7,kmeans_cluster_8,kmeans_cluster_9,kmeans_cluster_10,pca_1,ica_1,tsvd_1,grp_1,srp_1,pca_2,ica_2,tsvd_2,grp_2,srp_2,pca_3,ica_3,tsvd_3,grp_3,srp_3,pca_4,ica_4,tsvd_4,grp_

In [20]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, objective="regression_l1",
            boosting_type='dart',
            learning_rate=0.13110563910426518,
            max_depth=50,
            n_estimators=1000,
            num_leaves=200,
            reg_alpha=1.5921278701590583e-05,
            reg_lambda=1.0,
            subsample_for_bin=200000
)

model.fit(X_train, y_train)

print("Train MAE: ", mean_absolute_error(y_train, model.predict(X_train).abs()))
print("MeanAE: ", mean_absolute_error(y_valid, model.predict(X_valid).abs()))
print("MedianAE: ", median_absolute_error(y_valid, model.predict(X_valid).abs()))

Train MAE:  1.8148148381483926
MeanAE:  2.0142125091471437
MedianAE:  1.3562445642837888


In [21]:
sub_df = pd.DataFrame(X_test['DW_PET_VST_ID'])
sub_df['AgeEstimate'] = model.predict(X_test.drop('DW_PET_VST_ID', axis=1)).abs()
sub_df.head()

,DW_PET_VST_ID,AgeEstimate
0,76818738,1.762939
1,115189677,2.756423
2,140822898,3.315399
3,137576910,3.098501
4,68211647,8.602845


In [22]:
sub_df.to_csv("data/submission.csv", index=False)
sub_df.shape

(140844, 2)

In [23]:
save_memory(X_train)
save_memory(X_test)

if save_data:
    X_train.to_csv('data/X_train.csv', index=False)
    X_valid.to_csv('data/X_valid.csv', index=False)
    X_test.to_csv('data/X_test.csv', index=False)
    y_train.to_csv('data/y_train.csv', index=False)
    y_valid.to_csv('data/y_valid.csv', index=False)